# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
!pip install --upgrade pip
!pip install xgboost

    100% |████████████████████████████████| 1.5MB 6.1MB/s eta 0:00:01
  Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1
     |████████████████████████████████| 127.6 MB 11 kB/s s eta 0:00:01  |▋                               | 2.2 MB 3.2 MB/s eta 0:00:39     |███▏                            | 12.7 MB 3.2 MB/s eta 0:00:36     |████                            | 16.2 MB 3.2 MB/s eta 0:00:35     |█████▊                          | 22.8 MB 3.2 MB/s eta 0:00:33     |███████                         | 27.9 MB 34.4 MB/s eta 0:00:03     |███████▍                        | 29.5 MB 34.4 MB/s eta 0:00:03     |█████████████▋                  | 54.4 MB 34.4 MB/s eta 0:00:03     |██████████████                  | 55.8 MB 34.4 MB/s eta 0:00:03     |███████████████                 | 60.1 MB 24.3 MB/s eta 0:00:03     |███████████████▊                | 62.8 MB 24.3 MB/s eta 0:00:03     |█████████████████▌              | 69.7 MB 24.3 MB/s eta 0:00:03  

In [3]:
import xgboost as xgb

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import svm
import pickle
from sklearn.externals import joblib 

In [5]:
nltk.download('stopwords') # download for stopwords
nltk.download('wordnet') # download for lemmatization
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///Project_Response.db')
df = pd.read_sql_table('Data',engine)
X = df['message']
y = df.loc[:,'related':'direct_report']
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df.loc[:,'related':'direct_report'].sum(axis=0).sort_values(ascending=False)[:10].values

array([20252, 10841,  7286,  5064,  4464,  3441,  2917,  2452,  2440,  2308])

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    # Tokenize text after normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    words = word_tokenize(text.lower())
    # Remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # Apply a stemmer
    words = [PorterStemmer().stem(w) for w in words]
    return words

In [9]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [9]:
tokenize(X[0])

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [74]:
# pipeline = Pipeline([
#     ('features', FeatureUnion([

#         ('text_pipeline', Pipeline([
#             ('vect', CountVectorizer(tokenizer=tokenize)),
#             ('tfidf', TfidfTransformer())
#         ])),

#         ('starting_verb', StartingVerbExtractor())
#     ])),

#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
# ])

In [79]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42, n_jobs=-1)))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [95]:
# Training pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])

In [96]:
# Predicting labels
y_pred = pipeline.predict(X_test)

In [19]:
y_test.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
1968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17679,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5923,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1087,1,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7172,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
# Source of the error: child_alone only has 1 class (0)
precision_score(y_test['child_alone'], y_pred[:,9], average= 'weighted',labels=np.unique(y_pred))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


1.0

In [11]:
def avg_f1(y_test, y_pred):
    report = pd.DataFrame(columns=['label','f1'])
    for i,col in enumerate(y_test.columns):
        report.loc[i,'f1'] = f1_score(y_test[col], y_pred[:,idx], average= 'weighted')
    return report['f1'].mean()

In [12]:
def make_report(y_test, y_pred):
    report = pd.DataFrame(columns=['label','accuracy','precision','recall','f1'])
    for i,col in enumerate(y_test.columns):
        idx = y_test.columns.get_loc(col)
        report.loc[i,'label'] = col
        report.loc[i,'accuracy'] = accuracy_score(y_test[col], y_pred[:,idx])
        report.loc[i,'precision'] = precision_score(y_test[col], y_pred[:,idx], average= 'weighted',labels=np.unique(y_pred))
        report.loc[i,'recall'] = recall_score(y_test[col], y_pred[:,idx], average= 'weighted')
        report.loc[i,'f1'] = f1_score(y_test[col], y_pred[:,idx], average= 'weighted')
    return report


In [197]:
make_report(y_test, y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,label,accuracy,precision,recall,f1
0,related,0.815763,0.803333,0.815763,0.798867
1,request,0.89152,0.886984,0.89152,0.877692
2,offer,0.995162,0.990347,0.995162,0.992748
3,aid_related,0.779093,0.778126,0.779093,0.776194
4,medical_help,0.924242,0.904562,0.924242,0.897576
5,medical_products,0.9548,0.944807,0.9548,0.936002
6,search_and_rescue,0.972753,0.966087,0.972753,0.961652
7,security,0.979883,0.960417,0.979883,0.970053
8,military,0.967023,0.955144,0.967023,0.953959
9,child_alone,1,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [163]:
# Looking for parameters to modify
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f8b04343400>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
               oob_score=False, random_state=42, 

In [180]:
# Setting the values for the parameters
parameters = {
    #'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [30, 50],
    #'clf__estimator__min_samples_split': [2, 3],
}

scorer = make_scorer(avg_f1, greater_is_better= True)
# Setting up the gridsearch
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=4, scoring=scorer)

In [ ]:
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [192]:
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': (0.75, 1.0), 'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [30, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(avg_f1), verbose=4)

In [183]:
y_pred = cv.predict(X_test)

In [184]:
make_report(y_test,y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,label,accuracy,precision,recall,f1
0,related,0.815763,0.803333,0.815763,0.798867
1,request,0.89152,0.886984,0.89152,0.877692
2,offer,0.995162,0.990347,0.995162,0.992748
3,aid_related,0.779093,0.778126,0.779093,0.776194
4,medical_help,0.924242,0.904562,0.924242,0.897576
5,medical_products,0.9548,0.944807,0.9548,0.936002
6,search_and_rescue,0.972753,0.966087,0.972753,0.961652
7,security,0.979883,0.960417,0.979883,0.970053
8,military,0.967023,0.955144,0.967023,0.953959
9,child_alone,1,1,1,1


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [219]:
pipeline_xgb = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(xgb.XGBClassifier(random_state=42, n_jobs=-1)))])

In [235]:
pipeline_xgb.named_steps

{'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=<function tokenize at 0x7f8b04343400>, vocabulary=None),
 'tfidf': TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True),
 'clf': MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
        colsample_bynode=None, colsample_bytree=None, gamma=None,
        gpu_id=None, importance_type='gain', interaction_constraints=None,
        learning_rate=None, max_delta_step=None, max_depth=None,
        min_child_w..._pos_weight=None, subsample=None,
        tree_method=None, validate_parameters=None, verbosity=None),
            n_jobs=1)}

In [236]:
# Setting the values for the parameters
parameters = {
    #'vect__ngram_range': ((1, 1), (1, 2)),
    #'vect__max_df': (0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    #'tfidf__use_idf': (True, False),
    'clf__estimator__max_depth': [6, 8],
    'clf__estimator__min_child_weight': [1, 2],
}

cv_xgb = GridSearchCV(pipeline_xgb, param_grid=parameters, verbose=1, scoring=scorer)

In [ ]:
cv_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined an

In [223]:
y_pred = cv_xgb.predict(X_test)
# y_pred = model.predict(X_test)

In [18]:
make_report(y_test, y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,label,accuracy,precision,recall,f1
0,related,0.816272,0.804107,0.816272,0.799662
1,request,0.897122,0.89054,0.897122,0.889692
2,offer,0.995289,0.995311,0.995289,0.99306
3,aid_related,0.779221,0.77877,0.779221,0.77571
4,medical_help,0.92628,0.911667,0.92628,0.915017
5,medical_products,0.958365,0.950257,0.958365,0.952017
6,search_and_rescue,0.972116,0.963892,0.972116,0.965666
7,security,0.979119,0.965375,0.979119,0.970372
8,military,0.971352,0.966361,0.971352,0.967625
9,child_alone,1,1,1,1


### 9. Export your model as a pickle file

In [ ]:
joblib.dump(cv_xgb.best_estimator_, 'xgbclassifier.pkl')

In [13]:
model = joblib.load('xgbclassifier.pkl')

In [18]:
model.best_params_

{'clf__estimator__max_depth': 8, 'clf__estimator__min_child_weight': 1}

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.